In [378]:
import pandas as pd
from ast import literal_eval
from imdb import Cinemagoer
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image, UnidentifiedImageError
import requests
from io import BytesIO
import os
from joblib import Parallel, delayed
import re

In [379]:
def extract_cast(cast_str,num_cast_members = 3,lower = True, sort = True):
    cast_list = literal_eval(cast_str)
    
    
    num_cast_members = min(num_cast_members,len(cast_list))
    names = []
    for i in range(num_cast_members):
        name = cast_list[i]['name']
        if lower:
            name = name.lower().strip()
            
        names.append(name)
    return sorted(names) if sort else names

def extract_crew(crew_str,lower = True):
    crew_list=[]
    for i in literal_eval(crew_str):
        if i['job'] == 'Director':
            crew_list.append(i['name'].lower().strip() if lower else i['name'])
            break
    return crew_list


def extract_genres(genres_str,max_genres=3,sort = True, lower = True):
    genres_list = literal_eval(genres_str)
    max_genres = min(max_genres,len(genres_list))
    
    genre_names = []
    c = 1
    while c<=max_genres:
        g = genres_list[c-1]['name'].lower().strip() if lower else genres_list[c-1]['name']
        genre_names.append(g)
        c+=1
        
    return sorted(genre_names) if sort else genre_names
        
def extract_production_company(pc_str,num_companies = 4,lower = True, sort = True):
    company_list = literal_eval(pc_str)
    
    num_companies = min(num_companies,len(company_list))
    
    names = []
    for i in range(num_companies):
        name = company_list[i]['name']
        if lower:
            name = name.lower().strip()
        names.append(name)
    return sorted(names) if sort else names


def extract_languages(language_str,num_languages = 4,lower = True, sort = True):
    languages_list = literal_eval(language_str)
    
    num_languages = min(num_languages,len(languages_list))
    
    names = []
    for i in range(num_languages):
        name = languages_list[i]['iso_639_1']
        if lower:
            name = name.lower().strip()
        names.append(name)
    return sorted(names) if sort else names


def read_image_url(url):
    response = requests.get(url)
    img = np.array(Image.open(BytesIO(response.content)))
    return img



def download_image_url(url,file_path):
    
    response = requests.get(url)
    img = response.content

    if response.status_code==200:
        with open(file_path, 'wb') as handler:
            handler.write(img)
        return 'OK'

    return 'ERROR'

def download_poster(row,data_path = '',id_name = 'imdb_id',poster_col = 'poster_link'):
    
    file_path = os.path.join(data_path,
                             f"posters/{int(row[id_name])}.jpg")

    status = download_image_url(row[poster_col],
                   file_path
                  )
    if status == 'ERROR':
        return int(row[id_name])

In [380]:
meta_data = pd.read_csv('movies_metadata.csv')
credits = pd.read_csv('credits.csv')
poster_ratings = pd.read_csv('MovieGenre.csv',encoding = "ISO-8859-1")
title_type = pd.read_csv('oficial_imdb_dataset/title.basics.tsv',sep='\t')
title_type['imdb_id'] = title_type['tconst'].str.replace('tt','').astype(float)


meta_data_cols = [
                'title',
                'id',
                'imdb_id',
                'original_language',
                'original_title',
                'overview',
                #'poster_path',
                'production_companies',
                'budget',
                'genres',
                'production_countries',
                'release_date',
                'revenue',
                'runtime',
                'spoken_languages',
                    ]


meta_data['release_date'] = pd.to_datetime(meta_data['release_date'],errors='coerce')


#filters
mask1 = (meta_data['release_date'].dt.year>=1980)&\
        (meta_data['original_language']=='en')&\
        (meta_data['status']=='Released')&\
        (meta_data['genres']!='[]')

meta_data = meta_data[mask1]

meta_data['imdb_id'] = meta_data['imdb_id'].str.replace('tt','').astype(float)
meta_data['budget'] = meta_data['budget'].astype(float)
meta_data['id'] = meta_data['id'].astype(int)

meta_data = meta_data[meta_data_cols]

#Append title Type and filter only the movies
meta_data = meta_data.merge(title_type[['imdb_id','titleType']],how='left',on='imdb_id')
mask2 = (meta_data['titleType']=='movie')
meta_data = meta_data[mask2]

meta_data.drop_duplicates(subset=['imdb_id'],inplace=True)
poster_ratings.drop_duplicates(subset=['imdbId'],inplace=True)
credits.drop_duplicates(subset=['id'],inplace=True)


meta_data.reset_index(drop=True,inplace=True)
poster_ratings.reset_index(drop=True,inplace=True)
credits.reset_index(drop=True,inplace=True)

poster_ratings = poster_ratings[['imdbId','IMDB Score','Poster']]\
                            .rename(columns = {'imdbId':'imdb_id',
                                                'IMDB Score':'imdb_rating',
                                                'Poster':'poster_link'})
poster_ratings['imdb_id']= poster_ratings['imdb_id'].astype(float)

meta_data = meta_data.merge(poster_ratings,how = 'left',on='imdb_id')
meta_data[meta_data['overview'].apply(lambda x: len(x) if isinstance(x,str) else x)<=2]=np.nan
meta_data.dropna(subset = ['imdb_id','poster_link','overview','imdb_rating'],inplace=True,how = 'any')


meta_data.reset_index(drop=True,inplace=True)
poster_ratings.reset_index(drop=True,inplace=True)
credits.reset_index(drop=True,inplace=True)

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [384]:
NUM_CAST_MEMBERS = 3
NUM_GENRES = 3
NUM_PROD_COMPANIES = 4
NUM_LANGUAGES = 4

cast = credits['cast'].apply(lambda x: extract_cast(x,num_cast_members=NUM_CAST_MEMBERS)).tolist()
cast = pd.DataFrame(cast,columns = [f'cast_{i}'for i in range(1,NUM_CAST_MEMBERS+1)])

crew = credits['crew'].apply(extract_crew).tolist()
crew = pd.DataFrame(crew,columns = ['director'])

genres = meta_data['genres'].apply(lambda x: extract_genres(x,max_genres=NUM_GENRES)).tolist()
genres = pd.DataFrame(genres,columns = [f'genre_{i}'for i in range(1,NUM_GENRES+1)])

p_companies = meta_data['production_companies'].apply(lambda x: extract_production_company(x,num_companies=NUM_PROD_COMPANIES)).tolist()
p_companies = pd.DataFrame(p_companies,columns = [f'p_company_{i}'for i in range(1,NUM_PROD_COMPANIES+1)])

languages = meta_data['spoken_languages']\
    .apply(lambda x: extract_languages(x,num_languages=NUM_LANGUAGES))\
    .tolist()
languages = pd.DataFrame(languages,columns = [f'language_{i}'for i in range(1,NUM_LANGUAGES+1)])


In [385]:
#Merge all
credits = pd.concat([credits[['id']],crew,cast],axis=1)
meta_data = meta_data.merge(credits,on='id',how='left')
meta_data = pd.concat([meta_data,genres,p_companies,languages],axis=1)

In [ ]:
%%time
#DO NOT RUN NOW
errors = Parallel(n_jobs=-1)(delayed(download_poster)(row) for _,row in meta_data[['imdb_id','poster_link']].iterrows())
errors = list(filter(None,errors))



In [171]:
from re import sub
from decimal import Decimal
from bs4 import BeautifulSoup

class BoxOfficeMojoScraper:
    
    def __init__(self,start_year,end_year):
        self.start_year = start_year
        self.end_year = end_year
        self.col_names = ['rank',
                          'title',
                          'revenue worldwide',
                          'revenue domestic',
                          'domstic %',
                          'revenue foreign',
                          'foreign %']
        self.movies = []
        
    def __get_movies_in_year_objects(self,year):
        url = f'https://www.boxofficemojo.com/year/world/{year}/'
        page = requests.get(url)
        soup=BeautifulSoup(page.text,"html.parser")
        
        objects = soup.find_all("tr")
        return objects[1:]
    
    def __get_single_object_row(self,obj):
        items = obj.find_all('td')
        row = {}
        money_cols = ['revenue worldwide','revenue domestic','revenue foreign']
        for col,item in zip(self.col_names,items):

            data = item.text
            
            if data =='-':
                data = np.nan
            elif col in money_cols:
                data = float(Decimal(sub(r'[^\d.]', '', data)))
            row[col] = data
        return row

    def __get_all_objects_rows(self,objects):
        for o in objects:
            self.movies.append(self.__get_single_object_row(o))
        return self.movies
    
    def __get_movies_single_year(self,year):
        objects = self.__get_movies_in_year_objects(year)
        self.__get_all_objects_rows(objects)
    
    def extract_movies(self):
        for year in range(self.start_year,self.end_year+1):
            self.__get_movies_single_year(year)
            print(f'{year}: done!')
        return self.to_df()
    
    def to_df(self):
        return pd.DataFrame(self.movies)

In [172]:
bom = BoxOfficeMojoScraper(1980,2020)
box_office_data = bom.extract_movies()

1980: done!
1981: done!
1982: done!
1983: done!
1984: done!
1985: done!
1986: done!
1987: done!
1988: done!
1989: done!
1990: done!
1991: done!
1992: done!
1993: done!
1994: done!
1995: done!
1996: done!
1997: done!
1998: done!
1999: done!
2000: done!
2001: done!
2002: done!
2003: done!
2004: done!
2005: done!
2006: done!
2007: done!
2008: done!
2009: done!
2010: done!
2011: done!
2012: done!
2013: done!
2014: done!
2015: done!
2016: done!
2017: done!
2018: done!
2019: done!
2020: done!


In [173]:
box_office_data.to_csv('box_office_mojo_data.csv',index=False)

In [18]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import multiprocessing as mp

class TheNumbersScraper:
    
    def __init__(self,start_page=1,end_page=-1,num_movies_per_page=100):
        self.end_page = float('inf') if end_page==-1 else end_page
        self.start_page = start_page
        self.col_names = ['rank',
                          'year',
                          'title',
                          'revenue worldwide',
                          'revenue domestic',
                          'revenue international']
        self.movies = []
        self.driver = webdriver.Chrome('../../../../Desktop/Drivers/chromedriver')
        self.num_movies_per_page = num_movies_per_page
        
    def __get_movies_in_page_objects(self,page_num):
        url = f'https://www.the-numbers.com/box-office-records/worldwide/all-movies/cumulative/all-time/{page_num}'
        self.driver.get(url)
        table = self.driver.find_elements_by_tag_name('table')
        
        if table==[]:
            print("no more movies")
            return []
        
        table = table[0]
        objects = table.find_elements_by_tag_name('tr')
        return objects[1:]
    
    def __get_signle_object_row(self,obj):
        items = obj.find_elements_by_tag_name('td')
        row = {}
        money_cols = ['revenue worldwide','revenue domestic','revenue international']
        
        for col,item in zip(self.col_names,items):
            data = item.text
            if data =='':
                data = np.nan
            elif col in money_cols:
                data = float(Decimal(sub(r'[^\d.]', '', data)))
            elif col == 'year':
                data = int(data)
            row[col] = data
            
        return row

    def __get_all_objects_rows(self,objects):
        for o in objects:
            self.movies.append(self.__get_signle_object_row(o))
        return self.movies
    
    def __get_movies_single_page(self,page_num):
        objects = self.__get_movies_in_page_objects(page_num)
        self.__get_all_objects_rows(objects)
    
    def extract_movies(self):
        curr_page = self.start_page
        
        while curr_page<=self.end_page:
            objects = self.__get_movies_in_page_objects(curr_page)
            
            if objects==[]:
                break
                
            self.__get_all_objects_rows(objects)
            print(f'page {curr_page}: done!')
            
            curr_page+=self.num_movies_per_page
            
        return self.to_df()
    
    def to_df(self):
        return pd.DataFrame(self.movies)

In [20]:
def the_numbers_scraper_multiprocess(start_page,end_page):
    tns = TheNumbersScraper(start_page=start_page,end_page=end_page)
    m = tns.extract_movies()
    m['num_movies_range'] = f'{start_page}-{end_page}'
    return m

In [21]:
def get_batches(max_page_num,num_movies_per_page,n_jobs = 8):
    
    max_movies = max_page_num+num_movies_per_page
    pages_per_thread = max_movies//(n_jobs*num_movies_per_page)
    remeinder =  max_movies%(n_jobs*num_movies_per_page)
    movies_per_thread = pages_per_thread*num_movies_per_page
    
    batches = []
    for i in range(1,max_page_num,pages_per_thread*num_movies_per_page):
        batches.append([i,i+movies_per_thread-1])
    return batches


In [73]:
%%time
num_movies_per_page = 100
n_jobs = 8
max_page_num = 33701
batches = get_batches(max_page_num=max_page_num,
                      num_movies_per_page=num_movies_per_page,
                      n_jobs=n_jobs)
with mp.Pool(8) as p:
    the_numbers_df=p.starmap(the_numbers_scraper_multiprocess,batches)

page 12601: done!
page 4201: done!
page 25201: done!
page 16801: done!
page 1: done!
page 8401: done!
page 21001: done!
page 29401: done!
page 12701: done!
page 16901: done!
page 25301: done!
page 101: done!
page 4301: done!
page 8501: done!
page 21101: done!
page 29501: done!
page 12801: done!
page 17001: done!
page 25401: done!
page 201: done!
page 4401: done!
page 21201: done!
page 8601: done!
page 29601: done!
page 17101: done!
page 25501: done!
page 301: done!
page 12901: done!
page 4501: done!
page 21301: done!
page 8701: done!
page 29701: done!
page 13001: done!
page 17201: done!
page 401: done!
page 25601: done!
page 4601: done!
page 21401: done!
page 8801: done!
page 29801: done!
page 13101: done!
page 17301: done!
page 501: done!
page 25701: done!
page 4701: done!
page 21501: done!
page 8901: done!
page 29901: done!
page 13201: done!
page 17401: done!
page 601: done!
page 25801: done!
page 4801: done!
page 21601: done!
page 9001: done!
page 30001: done!
page 13301: done!
page

In [87]:
the_numbers_df = pd.concat(the_numbers_df)
the_numbers_df.reset_index(drop=True,inplace=True)
the_numbers_df.to_csv('the_numbers_df.csv',index=False)

In [389]:
the_numbers_df

,rank,year,title,revenue worldwide,revenue domestic,revenue international,num_movies_range
0,1,2009,Avatar,2.845900e+09,760507625.0,2.085392e+09,1-4200
1,2,2019,Avengers: Endgame,2.797801e+09,858373000.0,1.939428e+09,1-4200
2,3,1997,Titanic,2.207987e+09,659363944.0,1.548623e+09,1-4200
3,4,2015,Star Wars Ep. VII: The Force Awakens,2.064616e+09,936662225.0,1.127954e+09,1-4200
4,5,2018,Avengers: Infinity War,2.048360e+09,678815482.0,1.369544e+09,1-4200
...,...,...,...,...,...,...,...
33731,"33,732",2020,Before Everest,9.000000e+00,NaN,9.000000e+00,33601-37800
33732,"33,733",2015,Abulele,7.000000e+00,NaN,7.000000e+00,33601-37800
33733,"33,734",2019,Turbulencia Zombi,6.000000e+00,NaN,6.000000e+00,33601-37800
33734,"33,735",2017,ASYA,2.000000e+00,NaN,2.000000e+00,33601-37800


In [391]:
meta_data['year'] = meta_data['release_date'].dt.year
meta_data['month'] = meta_data['release_date'].dt.month
meta_data['day'] = meta_data['release_date'].dt.day

In [392]:
the_numbers_df.rename(columns = {'year':'the_numbers_year'},inplace=True)
meta_data = meta_data.merge(the_numbers_df.drop(['rank','num_movies_range'],axis=1),
                    on=['title'],
                    how ='left')

In [393]:
meta_data = meta_data[((meta_data['year']-meta_data['the_numbers_year']).abs().fillna(0)<=1)]
meta_data.drop_duplicates(subset='imdb_id',inplace=True)
meta_data.drop('the_numbers_year',axis=1,inplace=True)
meta_data.rename(columns = {'revenue worldwide':'revenue_worldwide_TN',
                             'revenue domestic':'revenue_domestic_TN',
                             'revenue international':'revenue_international_TN'},inplace=True)

In [331]:
class BoxOfficeMojoScraperV2:
    
    def __init__(self,imdb_id_list):
        self.imdb_id_list = imdb_id_list
        self.movies = []
        
    def __get_movies_objects(self,imdb_id):
        
        imdb_id = str(int(imdb_id)).zfill(7)
        url = f"https://www.boxofficemojo.com/title/tt{imdb_id}/"
        page = requests.get(url)
        soup=BeautifulSoup(page.text,"html.parser")
        table = soup.find("div",class_="a-section a-spacing-none mojo-performance-summary-table")
        objects = table.find_all("div")
        return objects
    
    def __get_single_movie_row(self,objects):
        row = {}
        for o in objects:
            col = re.findall("[a-zA-Z]+",o.find('span',class_="a-size-small").text.strip())[0]
            data = o.find('span',class_="a-size-medium a-text-bold").text.strip()
            
            if (data =='-')|(data=='–'):
                data = np.nan
            else:
                data = float(Decimal(sub(r'[^\d.]', '', data)))
                
            row[col] = data
        return row

    
    def extract_movies(self):
        tot_movies = len(self.imdb_id_list)
        c = 1
        for imdb_id in self.imdb_id_list:
            objects = self.__get_movies_objects(imdb_id)
            row = self.__get_single_movie_row(objects)
            row['imdb_id'] = imdb_id
            self.movies.append(row)
            
            completed_pct = c*100/tot_movies
            if (completed_pct%10==0):
                print(f'{completed_pct}% done!')
            c+=1
        return self.to_df()
    
    def to_df(self):
        return pd.DataFrame(self.movies)

In [312]:
#Get list of movie ids
imdb_ids_list = meta_data['imdb_id'].unique()

#Run scraper
bom2=BoxOfficeMojoScraperV2(imdb_ids_list)
box_office_mojo_data_v2 = bom2.extract_movies()

#Save dataframe
box_office_mojo_data_v2.reset_index(drop=True,inplace=True)
box_office_mojo_data_v2 = box_office_mojo_data_v2.dropna(subset=['Worldwide'])
box_office_mojo_data_v2.to_csv('box_office_mojo_data_v2.csv',index=False)

In [394]:
## Merge BOM data

#Rename Columns
box_office_mojo_data_v2.rename(columns = {i:f'revenue_{i.lower()}_BOM'for i in box_office_mojo_data_v2.columns[:3]},
                              inplace=True)

#Merge
meta_data = meta_data.merge(box_office_mojo_data_v2,how='left',on='imdb_id')

In [395]:
(meta_data[['revenue','revenue_worldwide_TN','revenue_worldwide_BOM']].sum(axis=1)>0).sum()

11177

In [402]:
d=(meta_data['revenue_worldwide_TN']-meta_data['revenue_worldwide_BOM']).abs()*100/meta_data['revenue_worldwide_BOM']

In [407]:
d.describe()

count     8407.000000
mean        63.437282
std       1431.749915
min          0.000000
25%          0.017337
50%          1.345660
75%         14.412097
max      97897.692597
dtype: float64

In [410]:
meta_data[d>14.4][['title','year','revenue','revenue_worldwide_TN','revenue_worldwide_BOM']]

,title,year,revenue,revenue_worldwide_TN,revenue_worldwide_BOM
6,Sabrina,1995,0.0,87100000.0,53696278.0
8,Sudden Death,1995,64350171.0,20214270.0,64350171.0
13,Nixon,1995,13681765.0,34668249.0,13681765.0
14,Cutthroat Island,1995,10017322.0,18517322.0,10017322.0
19,Money Train,1995,35431113.0,77224232.0,35431113.0
...,...,...,...,...,...
15323,The End of Time,2012,0.0,13056.0,4375.0
15331,Burn Burn Burn,2016,0.0,19923.0,28123.0
15336,I.T.,2016,0.0,1718502.0,2128301.0
15366,Interrogation,2016,0.0,2178.0,28184.0


In [414]:
(~meta_data['revenue_worldwide_TN'].isna()).sum()

8670

In [415]:
(~meta_data['revenue_worldwide_BOM'].isna()).sum()

10858

In [418]:
((meta_data['revenue_worldwide_BOM'].isna())&(~meta_data['revenue_worldwide_TN'].isna())).sum()

263

In [453]:
meta_data.columns

Index(['title', 'id', 'imdb_id', 'original_language', 'original_title',
       'overview', 'production_companies', 'budget', 'genres',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'titleType', 'imdb_rating', 'poster_link',
       'director', 'cast_1', 'cast_2', 'cast_3', 'genre_1', 'genre_2',
       'genre_3', 'p_company_1', 'p_company_2', 'p_company_3', 'p_company_4',
       'language_1', 'language_2', 'language_3', 'language_4', 'year', 'month',
       'day', 'revenue_worldwide_TN', 'revenue_domestic_TN',
       'revenue_international_TN', 'revenue_domestic_BOM',
       'revenue_international_BOM', 'revenue_worldwide_BOM'],
      dtype='object')

In [455]:
poster_ratings.head()

,imdb_id,imdb_rating,poster_link
0,114709.0,8.3,https://images-na.ssl-images-amazon.com/images...
1,113497.0,6.9,https://images-na.ssl-images-amazon.com/images...
2,113228.0,6.6,https://images-na.ssl-images-amazon.com/images...
3,114885.0,5.7,https://images-na.ssl-images-amazon.com/images...
4,113041.0,5.9,https://images-na.ssl-images-amazon.com/images...


In [459]:
meta_data['production_companies'].apply(extract_production_company)

0                                [pixar animation studios]
1        [interscope communications, teitler film, tris...
2                           [lancaster gate, warner bros.]
3                 [twentieth century fox film corporation]
4             [sandollar productions, touchstone pictures]
                               ...                        
15421                 [beyond infinity, titan productions]
15422                          [golden circle productions]
15423                                 [new world pictures]
15424                        [international film exchange]
15425             [south australian film corporation, the]
Name: production_companies, Length: 15426, dtype: object

In [460]:
meta_data['production_companies']

0           [{'name': 'Pixar Animation Studios', 'id': 3}]
1        [{'name': 'TriStar Pictures', 'id': 559}, {'na...
2        [{'name': 'Warner Bros.', 'id': 6194}, {'name'...
3        [{'name': 'Twentieth Century Fox Film Corporat...
4        [{'name': 'Sandollar Productions', 'id': 5842}...
                               ...                        
15421    [{'name': 'Titan Productions', 'id': 1582}, {'...
15422    [{'name': 'Golden Circle Productions', 'id': 8...
15423         [{'name': 'New World Pictures', 'id': 1950}]
15424    [{'name': 'International Film Exchange', 'id':...
15425    [{'name': 'South Australian Film Corporation, ...
Name: production_companies, Length: 15426, dtype: object

In [494]:
original_meta_data = pd.read_csv('movies_metadata.csv')
crerdits_original = pd.read_csv('credits.csv')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [495]:
original_meta_data['imdb_id'] = original_meta_data['imdb_id'].str.replace('tt','').astype(float)
original_meta_data = original_meta_data[original_meta_data['imdb_id'].isin(meta_data['imdb_id'].unique())]

In [496]:
original_meta_data['production_companies'].dropna().apply(extract_production_company)

0                                [pixar animation studios]
1        [interscope communications, teitler film, tris...
2                           [lancaster gate, warner bros.]
3                 [twentieth century fox film corporation]
4             [sandollar productions, touchstone pictures]
                               ...                        
40990                 [beyond infinity, titan productions]
40991                          [golden circle productions]
40992                                 [new world pictures]
40994                        [international film exchange]
40995             [south australian film corporation, the]
Name: production_companies, Length: 15434, dtype: object

In [497]:
original_meta_data.shape

(15434, 24)

In [498]:
meta_data.shape

(15426, 41)

In [499]:
meta_data.columns

Index(['title', 'id', 'imdb_id', 'original_language', 'original_title',
       'overview', 'production_companies', 'budget', 'genres',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'titleType', 'imdb_rating', 'poster_link',
       'director', 'cast_1', 'cast_2', 'cast_3', 'genre_1', 'genre_2',
       'genre_3', 'p_company_1', 'p_company_2', 'p_company_3', 'p_company_4',
       'language_1', 'language_2', 'language_3', 'language_4', 'year', 'month',
       'day', 'revenue_worldwide_TN', 'revenue_domestic_TN',
       'revenue_international_TN', 'revenue_domestic_BOM',
       'revenue_international_BOM', 'revenue_worldwide_BOM'],
      dtype='object')

In [509]:
t=meta_data.merge(original_meta_data[['imdb_id',
                                    'production_companies',
                                    'genres',
                                    'spoken_languages']],on='imdb_id')

In [550]:
pd.read_csv('oficial_imdb_dataset/title.crew.tsv',nrows=5,sep='\t')

,tconst,directors,writers
0,tt0000001,nm0005690,\N
1,tt0000002,nm0721526,\N
2,tt0000003,nm0721526,\N
3,tt0000004,nm0721526,\N
4,tt0000005,nm0005690,\N


In [559]:
of_names = pd.read_csv('oficial_imdb_dataset/name.basics.tsv',sep='\t')
of_genres = pd.read_csv('oficial_imdb_dataset/title.basics.tsv',sep='\t')
of_principals = pd.read_csv('oficial_imdb_dataset/title.principals.tsv.gz',sep='\t')

In [563]:
of_principals['imdb_id'] = of_principals['tconst'].str.replace('tt','').astype(float)

In [564]:
of_principals = of_principals[of_principals['imdb_id'].isin(meta_data['imdb_id'].unique())]

In [565]:
of_principals

,tconst,ordering,nconst,category,job,characters,imdb_id
282424,tt0035423,10,nm0107463,editor,\N,\N,35423.0
282425,tt0035423,1,nm0000212,actress,\N,"[""Kate McKay""]",35423.0
282426,tt0035423,2,nm0413168,actor,\N,"[""Leopold""]",35423.0
282427,tt0035423,3,nm0000630,actor,\N,"[""Stuart Besser""]",35423.0
282428,tt0035423,4,nm0005227,actor,\N,"[""Charlie McKay""]",35423.0
...,...,...,...,...,...,...,...
39216475,tt6071534,4,nm2538937,self,\N,"[""Self""]",6071534.0
39216476,tt6071534,5,nm0006149,composer,\N,\N,6071534.0
39216477,tt6071534,6,nm0030997,cinematographer,\N,\N,6071534.0
39216478,tt6071534,7,nm6668963,editor,\N,\N,6071534.0


In [567]:
of_df = of_principals.merge(of_names,how = 'left',on = 'nconst')

In [569]:
of_df['primaryName'] = of_df['primaryName'].str.lower().str.strip()

In [571]:
of_df = of_df.merge(meta_data,how='left',on='imdb_id')

In [580]:
actors = (of_df['category']=='actress')|(of_df['category']=='actor')


of_df['mask'] = (of_df.loc[actors,'primaryName']==of_df.loc[actors,'cast_1'])|\
(of_df.loc[actors,'primaryName']==of_df.loc[actors,'cast_2'])|\
(of_df.loc[actors,'primaryName']==of_df.loc[actors,'cast_3'])


In [600]:
test = of_df[actors].groupby('imdb_id').apply(lambda x: min(3,len(set(x['primaryName']))) ==  x['mask'].sum())

In [618]:
test.index[~test]

Float64Index([  76582.0,   79309.0,   80012.0,   80362.0,   80365.0,   80406.0,
                80427.0,   80455.0,   80461.0,   80485.0,
              ...
              5164412.0, 5168192.0, 5225338.0, 5240748.0, 5274066.0, 5312612.0,
              5434870.0, 5456798.0, 5459730.0, 5646136.0],
             dtype='float64', name='imdb_id', length=4762)

In [626]:
meta_data[meta_data['imdb_id']==80012.0][['cast_1','cast_2','cast_3']]

,cast_1,cast_2,cast_3
7636,billy greenlees,john hughes,robert buchanan


In [620]:
meta_data[meta_data['imdb_id']==79309.0]

,title,id,imdb_id,original_language,original_title,overview,production_companies,budget,genres,production_countries,...,language_4,year,month,day,revenue_worldwide_TN,revenue_domestic_TN,revenue_international_TN,revenue_domestic_BOM,revenue_international_BOM,revenue_worldwide_BOM
12119,Hot T-Shirts,90806.0,79309.0,en,Hot T-Shirts,In order to avoid the closure a bar the owner ...,[],0.0,"[{'id': 35, 'name': 'Comedy'}]","[{'iso_3166_1': 'US', 'name': 'United States o...",...,None,1980,3,1,NaN,NaN,NaN,NaN,NaN,NaN


In [627]:
of_df[of_df['imdb_id']==80012.0]

,tconst,ordering,nconst,category,job,characters,imdb_id,primaryName,birthYear,deathYear,...,year,month,day,revenue_worldwide_TN,revenue_domestic_TN,revenue_international_TN,revenue_domestic_BOM,revenue_international_BOM,revenue_worldwide_BOM,mask
78,tt0080012,1,nm0543429,actor,\N,"[""The Doctor""]",80012.0,tom mannion,\N,\N,...,1980,10,1,NaN,NaN,NaN,NaN,NaN,NaN,False
79,tt0080012,2,nm0123428,actor,\N,"[""Eddie the Driver""]",80012.0,eddie burt,\N,\N,...,1980,10,1,NaN,NaN,NaN,NaN,NaN,NaN,False
80,tt0080012,3,nm0218111,self,\N,"[""Self""]",80012.0,richard demarco,\N,\N,...,1980,10,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,tt0080012,4,nm1684618,actor,\N,"[""The Tramp""]",80012.0,alex mackenzie,\N,\N,...,1980,10,1,NaN,NaN,NaN,NaN,NaN,NaN,False
82,tt0080012,5,nm0287025,director,\N,\N,80012.0,bill forsyth,1946,\N,...,1980,10,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83,tt0080012,6,nm0876419,composer,\N,\N,80012.0,colin tully,1954,2021,...,1980,10,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84,tt0080012,7,nm0183533,cinematographer,\N,\N,80012.0,michael coulter,1952,\N,...,1980,10,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85,tt0080012,8,nm0332906,editor,\N,\N,80012.0,john gow,\N,\N,...,1980,10,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,tt0080012,9,nm0040668,production_designer,\N,\N,80012.0,adrienne atkinson,1954,\N,...,1980,10,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [624]:
of_df['category'].unique()

array(['editor', 'actress', 'actor', 'director', 'writer', 'producer',
       'composer', 'cinematographer', 'archive_footage', 'self',
       'production_designer', 'archive_sound'], dtype=object)

In [630]:
of_genres['titleType'].unique()

array(['short', 'movie', 'tvEpisode', 'tvSeries', 'tvShort', 'tvMovie',
       'tvMiniSeries', 'tvSpecial', 'video', 'videoGame', 'tvPilot'],
      dtype=object)

In [637]:
of_genres = of_genres[of_genres['titleType']=='movie']

In [641]:
of_df.columns

Index(['tconst', 'ordering', 'nconst', 'category', 'job', 'characters',
       'imdb_id', 'primaryName', 'birthYear', 'deathYear', 'primaryProfession',
       'knownForTitles', 'title', 'id', 'original_language', 'original_title',
       'overview', 'production_companies', 'budget', 'genres',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'titleType', 'imdb_rating', 'poster_link',
       'director', 'cast_1', 'cast_2', 'cast_3', 'genre_1', 'genre_2',
       'genre_3', 'p_company_1', 'p_company_2', 'p_company_3', 'p_company_4',
       'language_1', 'language_2', 'language_3', 'language_4', 'year', 'month',
       'day', 'revenue_worldwide_TN', 'revenue_domestic_TN',
       'revenue_international_TN', 'revenue_domestic_BOM',
       'revenue_international_BOM', 'revenue_worldwide_BOM', 'mask'],
      dtype='object')